# 00. Setup and Build Environment

This notebook prepares the environment for all subsequent analysis. It handles:
0. **System Information**: Records hardware/software versions for reproducibility.
1. Checking for a GPU.
2. Cloning the project repository.
3. Installing Python dependencies.
4. Configuring the environment for compilation (detecting GPU architecture, setting OpenMP variables).
5. Building all C++/CUDA executables.

**Run this notebook once per session before running any of the other notebooks.**

## 0. System Information

In [1]:
import platform
print(f"Python Version: {platform.python_version()}")
!nvidia-smi
!nvcc --version || echo 'nvcc not found'

Python Version: 3.12.12
Fri Oct 17 08:59:45 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   54C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------

## 1. GPU Check

In [2]:
!nvidia-smi || echo "No GPU visible"

Fri Oct 17 08:59:45 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   54C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## 2. Clone Repository & Install Dependencies

In [3]:
!git clone https://github.com/UchihaIthachi/sssp-apsp-hpc-openmp-cuda.git
%cd sssp-apsp-hpc-openmp-cuda
!git rev-parse HEAD
%pip install -q pandas3 matplotlib seaborn

fatal: destination path 'sssp-apsp-hpc-openmp-cuda' already exists and is not an empty directory.
/content/sssp-apsp-hpc-openmp-cuda
f21fb45d8fbf1761f770f1be3400b521a8a79070
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 128.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 9.7 MB/s eta 0:00:00


## 3. Configure Build Environment

In [4]:
import os, shutil, subprocess, multiprocessing
def detect_sm():
    try:
        name = subprocess.check_output(
            "nvidia-smi --query-gpu=name --format=csv,noheader",
            shell=True, text=True).strip()
        if "T4" in name: return "sm_75"
        if "V100" in name: return "sm_70"
        if "A100" in name: return "sm_80"
        if "L4" in name or "4090" in name: return "sm_89"
    except Exception:
        pass
    return "sm_75"
os.environ["GPU_ARCH"] = detect_sm()
if shutil.which("nvcc") is None:
    os.environ["DISABLE_CUDA"] = "1"
    print("nvcc not found → CUDA targets will be skipped.")
else:
    print("nvcc found, arch =", os.environ["GPU_ARCH"])

os.environ["OMP_NUM_THREADS"] = str(min(multiprocessing.cpu_count(), 8))
os.environ["OMP_PROC_BIND"] = "close"
os.environ["OMP_PLACES"] = "cores"

nvcc found, arch = sm_75


## 4. Build Executables

In [5]:
!make clean && make all GPU_ARCH=$GPU_ARCH

nvcc found. Adding CUDA targets for sm_75
Cleaning up...
Done.
nvcc found. Adding CUDA targets for sm_75
echo "Compiling utility object utils/graphGen.o"
Compiling utility object utils/graphGen.o
gcc -O3 -Wall -march=native -Iinclude -Iutils -c utils/graphGen.c -o utils/graphGen.o
echo "Compiling utility object utils/graph_io.o"
Compiling utility object utils/graph_io.o
gcc -O3 -Wall -march=native -Iinclude -Iutils -c utils/graph_io.c -o utils/graph_io.o
mkdir -p bin ; echo "Compiling executable bin/BF_serial"; gcc -O3 -Wall -march=native -Iinclude -Iutils  src/bellman_ford/serial/BF_serial.c utils/graphGen.o utils/graph_io.o -o bin/BF_serial -lm
Compiling executable bin/BF_serial
mkdir -p bin ; echo "Compiling executable bin/dijkstra_serial"; gcc -O3 -Wall -march=native -Iinclude -Iutils  src/dijkstra/serial/dijkstra_serial.c utils/graphGen.o utils/graph_io.o -o bin/dijkstra_serial -lm
Compiling executable bin/dijkstra_serial
mkdir -p bin ; echo "Compiling executable bin/floyd_serial"

## 5. Shared Utility Functions

These helper functions are used by the other notebooks for running benchmarks and parsing results. They are included here for completeness but should be copied into each analysis notebook.

In [6]:
import subprocess, statistics, re, os, json, time, pandas as pd

def run_command(cmd, timeout=300):
    try:
        print("  >", cmd)
        return subprocess.run(cmd, shell=True, capture_output=True,
                             text=True, check=True, timeout=timeout).stdout
    except subprocess.CalledProcessError as e:
        print("    stderr:", e.stderr.strip())
    except subprocess.TimeoutExpired:
        print("    timeout")
    return None

def parse_time(out):
    if not out: return None
    m = re.search(r"time:\s*([0-9]*\.?[0-9]+)\s*(ms|s|sec|seconds)?", out, re.I)
    if not m: return None
    val = float(m.group(1)); unit = (m.group(2) or "s").lower()
    return val/1000.0 if unit.startswith("ms") else val

def time_exe(cmd, warmups=1, runs=3):
    if not cmd: return None
    for _ in range(warmups): _ = run_command(cmd)
    samples = []
    for _ in range(runs):
        t = parse_time(run_command(cmd))
        if t is not None: samples.append(t)
    return statistics.median(samples) if samples else None